Intraclass correlation and cronbachs alpha are very related... 

In [1]:
import numpy as np
import pandas as pd
import pingouin as pg
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [2]:
path = "../data/appraisal/export_appraisal.csv"
df = pd.read_csv(path)
df.head()

,user_id,survey_id,survey_item_index,filename,has_reply,emotion_1,video_id,mix,emotion_1_id,emotion_2_id,intensity_level,reply_dim_Novelty,reply_dim_Pleasantness,reply_dim_Goal conduciveness,reply_dim_Urgency,reply_dim_Power,reply_dim_Norm compatibility
0,default,1d1439eb990b9fbdc99bf899dc7a7f9cb61a9a404e5677...,0,A200_int_v_3.mp4,1,interest_curiosity,A200,0,29,100,3.0,1,4,4,1,3,3
1,default,1d1439eb990b9fbdc99bf899dc7a7f9cb61a9a404e5677...,1,A102_exc_v_3.mp4,1,excitement_expectation,A102,0,36,100,3.0,1,5,5,1,3,5
2,default,1d1439eb990b9fbdc99bf899dc7a7f9cb61a9a404e5677...,2,A102_fea_p_2.mp4,1,fear,A102,0,10,100,2.0,3,1,1,2,2,1
3,default,1d1439eb990b9fbdc99bf899dc7a7f9cb61a9a404e5677...,3,A102_amu_v_3.mp4,1,amusement,A102,0,13,100,3.0,2,4,4,2,3,4
4,default,1d1439eb990b9fbdc99bf899dc7a7f9cb61a9a404e5677...,4,A207_pea_v_2.mp4,1,peacefulness_serenity,A207,0,25,100,2.0,1,2,2,2,2,2


In [3]:
df.columns

Index(['user_id', 'survey_id', 'survey_item_index', 'filename', 'has_reply',
       'emotion_1', 'video_id', 'mix', 'emotion_1_id', 'emotion_2_id',
       'intensity_level', 'reply_dim_Novelty', 'reply_dim_Pleasantness',
       'reply_dim_Goal conduciveness', 'reply_dim_Urgency', 'reply_dim_Power',
       'reply_dim_Norm compatibility'],
      dtype='object')

In [4]:
scales = [
    'reply_dim_Novelty',
    'reply_dim_Pleasantness',
    'reply_dim_Goal conduciveness',
    'reply_dim_Urgency',
    'reply_dim_Power',
    'reply_dim_Norm compatibility'
]

scales_df = df[scales]
scales_df

,reply_dim_Novelty,reply_dim_Pleasantness,reply_dim_Goal conduciveness,reply_dim_Urgency,reply_dim_Power,reply_dim_Norm compatibility
0,1,4,4,1,3,3
1,1,5,5,1,3,5
2,3,1,1,2,2,1
3,2,4,4,2,3,4
4,1,2,2,2,2,2
...,...,...,...,...,...,...
787,3,5,5,3,4,5
788,5,1,1,4,4,2
789,4,5,5,4,5,5
790,1,3,2,2,4,3


## ANOVA and Intraclass Correlation 

In [5]:
df_1 = df[df['survey_id'] == '1d1439eb990b9fbdc99bf899dc7a7f9cb61a9a404e5677631723cd56788b2d4a']
df_1

,user_id,survey_id,survey_item_index,filename,has_reply,emotion_1,video_id,mix,emotion_1_id,emotion_2_id,intensity_level,reply_dim_Novelty,reply_dim_Pleasantness,reply_dim_Goal conduciveness,reply_dim_Urgency,reply_dim_Power,reply_dim_Norm compatibility
0,default,1d1439eb990b9fbdc99bf899dc7a7f9cb61a9a404e5677...,0,A200_int_v_3.mp4,1,interest_curiosity,A200,0,29,100,3.0,1,4,4,1,3,3
1,default,1d1439eb990b9fbdc99bf899dc7a7f9cb61a9a404e5677...,1,A102_exc_v_3.mp4,1,excitement_expectation,A102,0,36,100,3.0,1,5,5,1,3,5
2,default,1d1439eb990b9fbdc99bf899dc7a7f9cb61a9a404e5677...,2,A102_fea_p_2.mp4,1,fear,A102,0,10,100,2.0,3,1,1,2,2,1
3,default,1d1439eb990b9fbdc99bf899dc7a7f9cb61a9a404e5677...,3,A102_amu_v_3.mp4,1,amusement,A102,0,13,100,3.0,2,4,4,2,3,4
4,default,1d1439eb990b9fbdc99bf899dc7a7f9cb61a9a404e5677...,4,A207_pea_v_2.mp4,1,peacefulness_serenity,A207,0,25,100,2.0,1,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,default,1d1439eb990b9fbdc99bf899dc7a7f9cb61a9a404e5677...,83,A102_neg_sur_p_2.mp4,1,negative_surprise,A102,0,11,100,2.0,3,2,1,2,2,2
84,default,1d1439eb990b9fbdc99bf899dc7a7f9cb61a9a404e5677...,84,A227_adm_p_3.mp4,1,admiration,A227,0,5,100,3.0,3,2,2,1,2,2
85,default,1d1439eb990b9fbdc99bf899dc7a7f9cb61a9a404e5677...,85,A200_sar_p_2.mp4,1,sarcasm,A200,0,31,100,2.0,1,2,2,1,2,3
86,default,1d1439eb990b9fbdc99bf899dc7a7f9cb61a9a404e5677...,86,A200_det_p_3.mp4,1,determination,A200,0,2,100,3.0,1,2,2,1,2,2


In [11]:
model = ols("reply_dim_Novelty ~ C(emotion_1,Treatment(reference='admiration'))", data=df_1)
result = model.fit()

# Print the summary
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:      reply_dim_Novelty   R-squared:                       0.739
Model:                            OLS   Adj. R-squared:                  0.485
Method:                 Least Squares   F-statistic:                     2.902
Date:                Fri, 26 Apr 2024   Prob (F-statistic):           0.000309
Time:                        16:49:06   Log-Likelihood:                -45.526
No. Observations:                  88   AIC:                             179.1
Df Residuals:                      44   BIC:                             288.1
Df Model:                          43                                         
Covariance Type:            nonrobust                                         
                                                                                  coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [17]:
model = ols("reply_dim_Novelty ~ C(emotion_1)", data=df_1)
result = model.fit()

# Print the summary
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:      reply_dim_Novelty   R-squared:                       0.739
Model:                            OLS   Adj. R-squared:                  0.485
Method:                 Least Squares   F-statistic:                     2.902
Date:                Fri, 26 Apr 2024   Prob (F-statistic):           0.000309
Time:                        16:50:38   Log-Likelihood:                -45.526
No. Observations:                  88   AIC:                             179.1
Df Residuals:                      44   BIC:                             288.1
Df Model:                          43                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

In [21]:
# Perform ANOVA
anova_results = sm.stats.anova_lm(result, typ=2)  # Type 2 ANOVA DataFrame
anova_results

,sum_sq,df,F,PR(>F)
C(emotion_1),41.125,43.0,2.902165,0.000309
Residual,14.500,44.0,NaN,NaN


By emotions

In [62]:
model = ols('reply_dim_Novelty ~ C(emotion_1) + C(survey_id)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
C(emotion_1),475.313131,43.0,11.862130,4.598533e-59
C(survey_id),209.093434,8.0,28.047989,3.080231e-38
Residual,689.573232,740.0,NaN,NaN


In [63]:
results = pg.intraclass_corr(data=df, targets='emotion_1', raters='survey_id', ratings='reply_dim_Novelty')
results

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.394606,6.866357,43,352,1.071705e-26,"[0.28, 0.53]"
1,ICC2,Single random raters,0.408699,10.634199,43,344,1.596623e-41,"[0.27, 0.56]"
2,ICC3,Single fixed raters,0.517017,10.634199,43,344,1.596623e-41,"[0.4, 0.65]"
3,ICC1k,Average raters absolute,0.854362,6.866357,43,352,1.071705e-26,"[0.78, 0.91]"
4,ICC2k,Average random raters,0.861509,10.634199,43,344,1.596623e-41,"[0.77, 0.92]"
5,ICC3k,Average fixed raters,0.905964,10.634199,43,344,1.596623e-41,"[0.86, 0.94]"


In [64]:
# Extract mean squares
MSB = anova_table['sum_sq']['C(emotion_1)'] / anova_table['df']['C(emotion_1)']  # Mean Square Between subjects (filenames)
MSE = anova_table['sum_sq']['Residual'] / anova_table['df']['Residual']  # Mean Square Error
MSR = anova_table['sum_sq']['C(survey_id)'] / anova_table['df']['C(survey_id)']  # Mean Square for Raters (survey_id)

# Number of raters and subjects
k = len(df['survey_id'].unique())  # number of raters
n = len(df['emotion_1'].unique())  # number of subjects

# Calculate ICC(2,k)
ICC_2k = (MSB - MSE) / (MSB + (k - 1) * MSE + (k * (MSR - MSE) / n))

print("ICC(2,k):", ICC_2k)

ICC(2,k): 0.4277326163288912


In [65]:
MSE

0.9318557193557195

By filenames

In [ ]:
scale = scales[0]

In [43]:
model = ols('reply_dim_Novelty ~ C(filename) + C(survey_id)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
C(filename),553.090909,87.0,7.232364,1.410710e-54
C(survey_id),209.093434,8.0,29.734004,4.479147e-40
Residual,611.795455,696.0,NaN,NaN


In [44]:
# Extract mean squares
MSB = anova_table['sum_sq']['C(filename)'] / anova_table['df']['C(filename)']  # Mean Square Between subjects (filenames)
MSE = anova_table['sum_sq']['Residual'] / anova_table['df']['Residual']  # Mean Square Error
MSR = anova_table['sum_sq']['C(survey_id)'] / anova_table['df']['C(survey_id)']  # Mean Square for Raters (survey_id)

# Number of raters and subjects
k = len(df['survey_id'].unique())  # number of raters
n = len(df['filename'].unique())  # number of subjects

# Calculate ICC(2,k)
ICC_2k = (MSB - MSE) / (MSB + (k - 1) * MSE + (k * (MSR - MSE) / n))

print("ICC(2,k):", ICC_2k)

ICC(2,k): 0.34298278431637114


In [57]:
anova_table['sum_sq']['Residual']

611.7954545454545

In [58]:
anova_table['df']['Residual']

696.0

In [45]:
results = pg.intraclass_corr(data=df, targets='filename', raters='survey_id', ratings='reply_dim_Novelty')
results

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.330961,5.452122,87,704,8.133354e-39,"[0.25, 0.42]"
1,ICC2,Single random raters,0.342983,7.232364,87,696,1.410710e-54,"[0.25, 0.45]"
2,ICC3,Single fixed raters,0.409153,7.232364,87,696,1.410710e-54,"[0.33, 0.5]"
3,ICC1k,Average raters absolute,0.816585,5.452122,87,704,8.133354e-39,"[0.75, 0.87]"
4,ICC2k,Average random raters,0.824508,7.232364,87,696,1.410710e-54,"[0.75, 0.88]"
5,ICC3k,Average fixed raters,0.861733,7.232364,87,696,1.410710e-54,"[0.81, 0.9]"


In [59]:
aov = pg.anova(data=df, dv="reply_dim_Novelty", between=["emotion_1", "survey_id"])
aov

,Source,SS,DF,MS,F,p-unc,np2
0,emotion_1,475.313131,43,11.053794,13.184646,2.577273e-53,0.588759
1,survey_id,209.093434,8,26.136679,31.175075,7.834299e-38,0.386428
2,emotion_1 * survey_id,357.573232,344,1.039457,1.239834,1.937937e-02,0.518543
3,Residual,332.000000,396,0.838384,NaN,NaN,NaN


In [48]:
msj = aov.at[1, "MS"]
msj

26.1366792929293

In [49]:
msb = aov.at[0, "MS"]
msb

6.357366771159874

In [60]:
mse = aov.at[2, "MS"]
mse

1.0394570707070698

ICC2k ska motsvara inverterad cronbachs alpha, petri rekommenderar denna. 